# Chattbot med RAG-teknik för frågor om Försäkringskassans ersättningar


### Import och konfiguration


In [ ]:
import os
import re
import numpy as np
import polars as pl
import unicodedata
from pypdf import PdfReader
from google import genai
from google.genai import types

In [ ]:
# Kontroll: ska embeddings byggas om?
REBUILD = False      # True = bygg om, False = ladda från fil
CHUNK_SIZE = 1000    # Hur många tecken varje chunk får vara
K_TOP = 20           # Hur många chunks hämtas vid fråga
EMBEDDING_MODEL = "text-embedding-004"
GENERATION_MODEL = "gemini-2.0-flash"

### Initiera Gemini-klienten


In [ ]:
# Startar Gemini-klienten via API-nyckel
client = genai.Client(api_key=os.getenv("API_KEY"))

### Läser in PDF-filer

In [ ]:
# Läser in all text från alla PDF-filer i en angiven mapp och slår ihop till en lång textsträng.
def read_pdfs_from_folder(folder_path):
    all_text = ""
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            path = os.path.join(folder_path, filename)
            reader = PdfReader(path)
            for page in reader.pages:
                all_text += page.extract_text()
    return all_text

# Normaliserar text och tar bort överflödiga mellanslag/radbrytningar.
def clean_text(text):
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(r'\s+', ' ', text)
    return text

# Läser in och förbehandlar all text från PDF-filer i 'data_pdf'-mappen.
raw_text = read_pdfs_from_folder("data_pdf")
text = clean_text(raw_text)

### Chunking 


In [ ]:
# Delar upp texten i mindre chunks baserat på meningar.
sentences = text.split(". ")
chunks = []
current_chunk = ""

for sentence in sentences:
    # Lägger till punkt om meningen saknar avslutningstecken.
    if not sentence.endswith(".") and not sentence.endswith("!") and not sentence.endswith("?"):
        sentence += "."

    # Bygger upp en chunk tills maxlängd nås
    if len(current_chunk) + len(sentence) + 1 <= CHUNK_SIZE:
        if current_chunk:
            current_chunk += " " + sentence
        else:
            current_chunk = sentence
    else:
        # Spara den nuvarande chunken och börja på en ny.
        chunks.append(current_chunk.strip())
        current_chunk = sentence

# Lägg till sista chunken om det finns något kvar.
if current_chunk:
    chunks.append(current_chunk.strip())

print(f"Antal chunks: {len(chunks)}")

### Embeddings



In [ ]:
# Skapar en embedding-vektor för en given textsträng, för att kunna jämföra och söka bland texter semantiskt.
def create_embedding(text: str, model=EMBEDDING_MODEL, task_type="RETRIEVAL_DOCUMENT") -> list[float]:
    response = client.models.embed_content(
        model=model,
        contents=text,
        config=types.EmbedContentConfig(task_type=task_type)
    )
    emb = response.embeddings[0].values
    v = np.array(emb)
    return (v / np.linalg.norm(v)).tolist()

In [ ]:
# Laddar embeddings från fil om de finns, annars skapas de och sparas.
# Returnerar både chunks och embeddings.
def load_or_build_embeddings(chunks: list[str]) -> list[list[float]]:
    # Om embeddings redan finns sparade på disk (och REBUILD=False), läs in dem
    if not REBUILD and os.path.exists("embeddings.parquet"):
        df = pl.read_parquet("embeddings.parquet")
        return df["texts"].to_list(), df["vectors"].to_list()
    # Annars, skapa embeddings från grunden och spara till fil för framtida bruk
    embeddings = [create_embedding(chunk) for chunk in chunks]
    df = pl.DataFrame({"texts": chunks, "vectors": embeddings})
    df.write_parquet("embeddings.parquet")
    return chunks, embeddings

### VectorStore


In [ ]:
class VectorStore:
    # Klass för att lagra text-chunks och deras embeddings (vektorer) för semantisk sökning med cosine similarity.
    def __init__(self, texts=None, vectors=None):
        # texts: lista med text-chunks
        # vectors: lista med embeddings för respektive text
        self.texts = texts if texts is not None else []
        self.vectors = [np.array(v) for v in vectors] if vectors is not None else []

    def add(self, vector, text):
        # Lägger till en embedding-vektor och motsvarande text-chunk
        self.vectors.append(np.array(vector))
        self.texts.append(text)

    def semantic_search(self, query_embedding, k=K_TOP):
        # Söker fram de text-chunks som liknar frågan mest enligt semantisk likhet.
        similarities = [
            (i, np.dot(query_embedding, v) / (np.linalg.norm(query_embedding) * np.linalg.norm(v)))
            for i, v in enumerate(self.vectors)
        ]
        similarities.sort(key=lambda x: x[1], reverse=True)
        k = min(k, len(self.texts))
        return [self.texts[i] for i, _ in similarities[:k]]

In [ ]:
# Skapar ett VectorStore-objekt och fyller det med alla text-chunks och deras embeddings.
vs = VectorStore()
for chunk, emb in zip(chunks, embeddings):
    vs.add(emb, chunk) # Lägger in varje embedding och dess text i VectorStore

print(f"Antal chunks i VectorStore: {len(vs.texts)}")

### Sökning och prompt till Gemini


In [ ]:
# Systemprompt som styr svarsstil och begränsningar
system_prompt = """
Du är expert på socialförsäkringsregler. Svara kortfattat och tydligt på den fråga som ställs och fokusera på det som efterfrågas.
Om frågan gäller hur länge man kan få en ersättning, svara i antal dagar om det finns sådan information i källtexten.
Om frågan gäller åldersgränser eller andra villkor, nämn dem punktvis om det behövs.
Om något är oklart eller saknas i källtexten men frågan gäller socialförsäkringar, skriv: 'Det framgår inte.'
Om frågan inte gäller Försäkringskassan eller socialförsäkringar, svara exakt: 'Det vet jag inte.'
Hitta inte på egna fakta.
"""
# Bygger prompt till modellen: sätter ihop frågan och de mest relevanta textbitarna
user_prompt = f"Fråga: {query}\n\nKONTEXT:\n" + "\n".join(results)

### Valideringsdata och utvärderingsprompt


In [ ]:
# Här skapas frågor och förväntade svar som används för automatisk utvärdering av chattbotens prestation.
validation_data = [
    {
        "question": "Vad gäller för sjukpenning för enskild firma?",
        "ideal_answer": [
            "Du kan få sjukpenning om du inte kan arbeta på grund av sjukdom och förlorar inkomst. Ersättningen är ca 80 % av din SGI, max 1 250 kr/dag. Du väljer själv karenstid (1, 7, 14, 30, 60 eller 90 dagar). Efter 364 dagar sänks ersättningen till 75 % av SGI. Du måste vara försäkrad i Sverige."
        ]
    },
    {
        "question": "Vilka får bostadsbidrag?",
        "ideal_answer": [
            "Barnfamiljer och unga under 29 år med låg inkomst och tillräcklig boendekostnad kan få bostadsbidrag. Maxbeloppet för unga utan barn är 1 300 kr/månad. Du måste vara folkbokförd där du bor."
        ]
    },
    {
        "question": "Vad gäller för bostadsbidrag till unga vuxna?",
        "ideal_answer": [
            "Unga under 29 år kan få bostadsbidrag om de har låg inkomst och boendekostnaden är tillräckligt hög. Bostadsyta max 60 kvm. Maxbeloppet är 1 300 kr/månad. Bidraget gäller inte inneboende utan barn."
        ]
    },
    {
        "question": "När sänks sjukpenningen från 80 till 75 procent?",
        "ideal_answer": [
            "Efter 364 dagar sänks sjukpenningen från 80 till 75 procent av SGI. Undantag kan finnas vid allvarlig sjukdom."
        ]
    },
    {
        "question": "Hur länge kan man få föräldrapenning?",
        "ideal_answer": [
            "Föräldrapenning kan tas ut i upp till 480 dagar per barn. 390 dagar är på sjukpenningnivå, 90 dagar på lägstanivå (180 kr/dag). Minst 90 dagar är reserverade för varje förälder."
        ]
    },
    {
        "question": "Kan jag få bostadsbidrag om jag är inneboende?",
        "ideal_answer": [
            "Barnfamiljer kan få bostadsbidrag även som inneboende. Unga utan barn kan inte få bostadsbidrag om de är inneboende."
        ]
    },
    {
        "question": "Får man sjukpenning om man reser utomlands?",
        "ideal_answer": [
            "Du kan behålla sjukpenning vid resa inom EU/EES eller Schweiz utan ansökan. För länder utanför EU/EES/Schweiz måste du ansöka innan resan. Resan får inte försämra din rehabilitering."
        ]
    },
    {
        "question": "Vad är huvudstaden i Tyskland?",
        "ideal_answer": [
            "Det vet jag inte."
        ]
    }
]

# Systemprompt till modellen för automatisk utvärdering av svaren.
evaluation_system_prompt = """
Du är ett strikt utvärderingssystem. För varje fråga:
- Ge 1 poäng om svaret är korrekt och täcker de viktigaste punkterna i något av de godkända svaren, även om det är utförligare eller mer pedagogiskt formulerat.
- Ge 0.5 poäng om svaret är delvis korrekt eller missar något väsentligt.
- Ge 0 poäng om svaret är felaktigt eller saknar viktiga delar.
- Skriv "Poäng: X" (X=1, 0.5, 0) på första raden.
- Motivera mycket kort på nästa rad.
"""

### Utvärdering och resultat


In [ ]:
# Loopa igenom valideringsfrågorna, generera svar med modellen och utvärdera poäng automatiskt.
total_score = 0  # Räknar ut totalpoängen

for item in validation_data:
    # Plocka ut fråga och idealsvar för aktuell valideringsrunda
    query = item["question"]
    expected_list = item["ideal_answer"]

    # Skapa embedding för frågan och hitta de mest relevanta text-chunks
    query_embedding = create_embedding(query)
    results = vs.semantic_search(query_embedding, k=K_TOP)
    user_prompt = f"Fråga: {query}\n\nKONTEXT:\n" + "\n".join(results)

    try:
        # Generera svar från modellen baserat på sökt kontext och systemprompt
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            config=types.GenerateContentConfig(system_instruction=system_prompt),
            contents=user_prompt
        )

        # Skapa en utvärderingsprompt där modellen får jämföra sitt eget svar mot idealsvar
        evaluation_prompt = (
            f"Fråga: {query}\n"
            f"AI-assistentens svar: {response.text}\n"
            f"Godkända svar är:\n- " + "\n- ".join(expected_list)
        )

        # Modellens svar utvärderas av en särskild utvärderingsprompt (auto-betyg)
        evaluation_response = client.models.generate_content(
            model="gemini-2.0-flash",
            config=types.GenerateContentConfig(system_instruction=evaluation_system_prompt),
            contents=evaluation_prompt
        )

        # Skriv ut fråga, svar och utvärdering för varje test
        print("\n--- EVALUERING ---")
        print(f"Fråga: {query}")
        print(f"Svar från modellen: {response.text}")
        print("Utvärdering:")
        print(evaluation_response.text)

        # Extrahera poäng från utvärderingssvaret och summera
        for line in evaluation_response.text.split("\n"):
            if "Poäng:" in line:
                try:
                    score = float(line.split(":")[-1].strip())
                    total_score += score
                except Exception as e:
                    print(f"Fel vid poängextraktion: {e}")

    except Exception as e:
        # Fångar och skriver ut fel som kan uppstå under utvärderingen
        print(f"Fel vid utvärdering av fråga '{query}': {e}")

# Skriv ut totalpoäng efter att alla frågor testats
print(f"\nTotalpoäng: {total_score} av {len(validation_data)} möjliga")

### Reflektion

Mitt projekt bygger på erfarenheter från Försäkringskassan, där jag märkt att viktig information på webbplatsen ofta är svår att hitta och förstå. För att göra denna information mer tillgänglig har jag utvecklat en chattbot som besvarar frågor om några vanliga ersättningar.

Chattboten använder Retrieval-Augmented Generation (RAG) för att kombinera språkmodellens naturliga språkförståelse med en lokal faktabas baserad på PDF-dokument från Försäkringskassans hemsida. Detta minskar risken för hallucinationer och ökar faktakontrollen.

Som exempel fokuserar jag på de tre vanligaste förmånerna: bostadsbidrag, föräldrapenning och sjukpenning. Eftersom det saknas öppna API:er och automatiserad datainsamling kan vara juridiskt och etiskt tveksamt, har jag manuellt samlat in och strukturerat informationen i PDF-format. Lösningen är flexibel och kan vidareutvecklas genom att utöka faktabasen med fler förmåner eller anpassas till andra informationsområden.

**Möjligheter (affärsmässiga och praktiska):**
- Gör information lättillgänglig och minskar belastning på kundtjänst.
- Ger myndigheter och företag möjlighet till kostnadsbesparingar genom automatiserade svar.
- Flexibel och återanvändbar lösning som kan anpassas för andra myndigheter, företag eller ämnesområden.
- Kan användas även för andra ämnen eller som internt stöd.
- Ger snabbare service och ökad tillgänglighet för användarna.

**Utmaningar:**
- Användare måste förstå att chattboten inte ersätter personlig rådgivning eller handläggare.
- Faktabasen måste hållas uppdaterad och korrekt; gamla dokument kan ge felaktiga svar.
- Det finns viss risk att svar tolkas som myndighetsbeslut, vilket kan få konsekvenser för användaren.

**Förbättringsförslag:**
- Utökat stöd för följdfrågor och förbättrad användarupplevelse.
- Förbättra gränssnittet med fler visuella element, till exempel tydligare kategorier eller hjälpfunktioner.
- Automatiskt hämta och visa länkar från de PDF-dokument som används som faktabas, för bättre källhänvisning.
- Vidareutveckla med stöd för fler språk och automatisk uppdatering av faktabasen.

**Etik:**
-Viktigt att tydligt informera användare om botens begränsningar och att den endast ger vägledande svar.
- Boten svarar endast på generella frågor om ersättningar baserat på öppen information.

**Sammanfattning:**  
En RAG-chattbot är ett kraftfullt verktyg för snabb, faktabaserad kundtjänst och ökad tillgänglighet.  Med fortsatt utveckling och regelbundna uppdateringar har tekniken potential att göra stor samhällsnytta. Det är dock viktigt att tydligt förklara botens begränsningar så att användaren inte missförstår informationen.
